In [1]:
import pandas as pd

# すべてのCSVファイル名をリストにまとめる
csv_files = [f'statistics_data{i}.csv' for i in range(1, 11)]

# 各CSVファイルを読み込み、データフレームをリストに保存
dataframes = [pd.read_csv(file) for file in csv_files]

# すべてのデータフレームを縦に連結
combined_df = pd.concat(dataframes, ignore_index=True)

# Modelの順序を保持するためにカテゴリー型に変更
combined_df['Model'] = pd.Categorical(combined_df['Model'], categories=combined_df['Model'].unique(), ordered=True)

# モデルごとにグループ化して集計
grouped_df = combined_df.groupby('Model', sort=False).agg({
    'Average Test Score': ['mean', 'std']
}).reset_index()

# カラム名のリネーム
grouped_df.columns = ['Model', 'Average Test Score Mean', 'Average Test Score Std']

# 小数点2位までに丸める
grouped_df['Average Test Score Mean'] = grouped_df['Average Test Score Mean'].round(2)
grouped_df['Average Test Score Std'] = grouped_df['Average Test Score Std'].round(2)

# モデルごとにデータを分割
rbf_df = grouped_df[grouped_df['Model'].str.startswith('RBF')]
linear_df = grouped_df[grouped_df['Model'].str.startswith('LINEAR')]
old_df = grouped_df[grouped_df['Model'].str.startswith('OLD')]

# 各データフレームのインデックスをリセット
rbf_df.reset_index(drop=True, inplace=True)
linear_df.reset_index(drop=True, inplace=True)
old_df.reset_index(drop=True, inplace=True)

# データを横に並べる
final_df = pd.concat([rbf_df, linear_df, old_df], axis=1)

# 結果を表示
print(final_df)

# CSVファイルとして保存
final_df.to_csv('grouped_statistics.csv', index=False)



  Model  Average Test Score Mean  Average Test Score Std    Model  \
0   RBF                    76.12                    0.61   LINEAR   
1  RBF2                    77.72                    0.86  LINEAR2   

   Average Test Score Mean  Average Test Score Std Model  \
0                    67.95                    0.53   OLD   
1                    77.02                    1.86   NaN   

   Average Test Score Mean  Average Test Score Std  
0                    60.85                    0.21  
1                      NaN                     NaN  


In [2]:
import pandas as pd

# すべてのCSVファイル名をリストにまとめる
csv_files = [f'statistics_data{i}.csv' for i in range(1, 11)]

# 各CSVファイルを読み込み、データフレームをリストに保存
dataframes = [pd.read_csv(file) for file in csv_files]

# すべてのデータフレームを縦に連結
combined_df = pd.concat(dataframes, ignore_index=True)

# Modelの順序を保持するためにカテゴリー型に変更
combined_df['Model'] = pd.Categorical(combined_df['Model'], categories=combined_df['Model'].unique(), ordered=True)

# 必要なカラムを選択
selected_columns = ['Model', 'Average TNR', 'Average TPR', 'Average Test Score']
combined_df = combined_df[selected_columns]

# モデルごとにデータを分割してリストに保存
models = combined_df['Model'].unique()
model_dfs = [combined_df[combined_df['Model'] == model].reset_index(drop=True) for model in models]

# 各モデルごとにデータフレームを作成し、リストに追加
final_dfs = []
for df in model_dfs:
    model_name = df['Model'].iloc[0]
    avg_tnr_mean = round(df['Average TNR'].mean(), 2)
    avg_tpr_mean = round(df['Average TPR'].mean(), 2)
    avg_test_score_mean = round(df['Average Test Score'].mean(), 2)
    avg_test_score_std = round(df['Average Test Score'].std(), 2)
    
    summary = pd.DataFrame({
        'Model': [model_name + '_MEAN'],
        'Average TNR': [avg_tnr_mean],
        'Average TPR': [avg_tpr_mean],
        'Average Test Score': [avg_test_score_mean],
        'Average Test Score Std': [avg_test_score_std]
    })
    
    df['Average Test Score Std'] = pd.NA  # 標準偏差の列を追加してNaNで埋める
    summary['Average Test Score Std'] = avg_test_score_std  # 標準偏差の列を更新
    
    df = df.drop(columns=['Model'])
    df.columns = [f'{model_name} {col}' for col in df.columns]
    
    summary.columns = [f'{model_name} {col}' for col in summary.columns]
    
    df = pd.concat([df, summary], ignore_index=True)
    
    final_dfs.append(df)

# すべてのモデルのデータフレームを横に連結
final_df = pd.concat(final_dfs, axis=1)

# 結果を表示
print(final_df)

# CSVファイルとして保存
final_df.to_csv('grouped_statistics.csv', index=False)



    RBF Average TNR  RBF Average TPR  RBF Average Test Score  \
0             84.43            67.57                   76.00   
1             83.60            69.27                   76.43   
2             84.43            67.57                   76.00   
3             80.83            69.63                   75.23   
4             82.97            69.23                   76.10   
5             85.33            69.27                   77.30   
6             81.30            69.07                   75.18   
7             83.10            69.77                   76.43   
8             84.07            68.37                   76.22   
9             83.40            69.20                   76.30   
10            83.35            68.90                   76.12   

   RBF Average Test Score Std RBF Model  LINEAR Average TNR  \
0                        <NA>       NaN               91.03   
1                        <NA>       NaN               86.63   
2                        <NA>       NaN   